<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/Simulationofanoilstabilizationprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulation of an oil stabilization process
In this notebook we will establish code for using NeqSim for simulation of an oil stabilization process

In [1]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *

# Process flow diagram

In [2]:
inputdata = {
  'feedFlowRateWell': 10,
  'wellPressure': 180.0,
  'wellTemperature':100.0,
  'topsidePressure': 90.0,
  'topsideTemperature': 5.0,
  'firstStagePressure': 75.0, 
  'temperatureOilHeater' : 75.9,
  'secondStagePressure': 8.6,
  'thirdStagePressure': 1.9,
  'firstStageSuctionCoolerTemperature': 25.3, 
  'secondStageSuctionCoolerTemperature': 24.5, 
  'thirdStageSuctionCoolerTemperature':25.0,
  'firstStageExportCoolerTemperature': 25.3, 
  'secondStageExportCoolerTemperature': 24.5
}

In [3]:
from neqsim.thermo import fluid, printFrame
from neqsim.process import compressor, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle2, setpoint, calculator

clearProcess()

wellFluid = fluid('pr')
wellFluid.addComponent("nitrogen", 0.08)
wellFluid.addComponent("CO2", 3.56)
wellFluid.addComponent("methane", 87.36)
wellFluid.addComponent("ethane", 4.02)
wellFluid.addComponent("propane", 1.54)
wellFluid.addComponent("i-butane", 0.2)
wellFluid.addComponent("n-butane", 0.42)
wellFluid.addComponent("i-pentane", 0.15)
wellFluid.addComponent("n-pentane", 0.20)
		
wellFluid.addTBPfraction("C6", 0.24, 84.99/1000.0, 695.0/1000.0)
wellFluid.addTBPfraction("C7", 0.34, 97.87/1000.0, 718.0/1000.0)
wellFluid.addTBPfraction("C8", 0.33, 111.54/1000.0, 729.0/1000.0)
wellFluid.addTBPfraction("C9", 0.19, 126.1/1000.0, 749.0/1000.0)
wellFluid.addTBPfraction("C10", 0.15, 140.14/1000.0, 760.0/1000.0)
wellFluid.addTBPfraction("C11", 0.69, 175.0/1000.0, 830.0/1000.0)
wellFluid.addTBPfraction("C12", 0.5, 280.0/1000.0, 914.0/1000.0)
wellFluid.addTBPfraction("C13", 0.103, 560.0/1000.0, 980.0/1000.0)

wellFluid.setMixingRule(2)
wellFluid.init(0)

wellFluid.setMolarComposition([0.08, 3.56, 87.36, 4.02, 1.54, 0.2, 0.42, 0.15, 0.2, 0.24, 0.34, 0.33, 0.19, 0.15, 0.69, 0.5, 0.03])

wellStream = stream(wellFluid)
wellStream.setName("dry well stream")
wellStream.setFlowRate(inputdata['feedFlowRateWell'], "MSm3/day")
wellStream.setTemperature(inputdata['wellTemperature'], "C")
wellStream.setPressure(inputdata['wellPressure'], "bara")

saturatedFeedGas = saturator(wellStream)
saturatedFeedGas.setName("water saturator")

waterSaturatedFeedGas = stream(saturatedFeedGas.getOutStream())
waterSaturatedFeedGas.setName("water saturated feed gas")

feedTPsetter = heater(waterSaturatedFeedGas)
feedTPsetter.setName('inletTP')
feedTPsetter.setOutPressure(inputdata['topsidePressure'], "bara")
feedTPsetter.setOutTemperature(inputdata['topsideTemperature'], "C")

chokeValve = valve(feedTPsetter.getOutStream())
chokeValve.setOutletPressure(inputdata['firstStagePressure'], 'bara')

feedToOffshoreProcess = stream(chokeValve.getOutStream())
feedToOffshoreProcess.setName("feed to offshore")

firstStageSeparator = separator3phase(feedToOffshoreProcess)
firstStageSeparator.setName("1st stage separator")

oilHeaterFromFirstStage = heater(firstStageSeparator.getOilOutStream())
oilHeaterFromFirstStage.setName("oil heater second stage")
oilHeaterFromFirstStage.setOutTemperature(inputdata['temperatureOilHeater'],'C')

oilThrotValve = valve(oilHeaterFromFirstStage.getOutStream())
oilThrotValve.setName("valve oil from first stage")
oilThrotValve.setOutletPressure(inputdata['secondStagePressure'])

secondStageSeparator = separator3phase(oilThrotValve.getOutStream())
secondStageSeparator.setName("2nd stage separator")

oilThrotValve2 = valve(secondStageSeparator.getOilOutStream())
oilThrotValve2.setName("valve oil from second stage")
oilThrotValve2.setOutletPressure(inputdata['thirdStagePressure'])

thirdStageSeparator = separator3phase(oilThrotValve2.getOutStream())
thirdStageSeparator.setName("3rd stage separator")

oilThirdStageToSep =  wellStream.clone()
oilThirdStageToSep.setName("resyc oil")
thirdStageSeparator.addStream(oilThirdStageToSep)

stableOil = stream(thirdStageSeparator.getOilOutStream())
stableOil.setName("stable oil")

stableOilPump = pump(stableOil,15.0,"stable oil pump")

firstStageCooler = cooler(thirdStageSeparator.getGasOutStream())
firstStageCooler.setName("1st stage cooler")
firstStageCooler.setOutTemperature(inputdata['firstStageSuctionCoolerTemperature'],'C')

firstStageScrubber = separator(firstStageCooler.getOutStream())
firstStageScrubber.setName("1st stage scrubber")

firstStageCompressor = compressor(firstStageScrubber.getGasOutStream())
firstStageCompressor.setName("1st stage compressor")
firstStageCompressor.setOutletPressure(inputdata['secondStagePressure'])
firstStageCompressor.setIsentropicEfficiency(0.75)

secondStageCooler = cooler(firstStageCompressor.getOutStream())
secondStageCooler.setName("2nd stage cooler")
secondStageCooler.setOutTemperature(inputdata['secondStageSuctionCoolerTemperature'],'C')

secondStageScrubber = separator(secondStageCooler.getOutStream())
secondStageScrubber.setName("2nd stage scrubber")

secondStageCompressor = compressor(secondStageScrubber.getGasOutStream())
secondStageCompressor.setName("2nd stage compressor")
secondStageCompressor.setOutletPressure(inputdata['firstStagePressure'])
secondStageCompressor.setIsentropicEfficiency(0.75)

richGasMixer = mixer("fourth Stage mixer")
richGasMixer.addStream(secondStageCompressor.getOutStream())
richGasMixer.addStream(firstStageSeparator.getGasOutStream())

dewPointControlCooler = cooler(richGasMixer.getOutStream())
dewPointControlCooler.setName("dew point cooler")
dewPointControlCooler.setOutTemperature(inputdata['thirdStageSuctionCoolerTemperature'],'C')

dewPointScrubber = separator(dewPointControlCooler.getOutStream())
dewPointScrubber.setName("dew point scrubber")

lpLiqmixer = mixer("LP liq gas mixer");
lpLiqmixer.addStream(firstStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(secondStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(dewPointScrubber.getLiquidOutStream());

lpResycle = recycle2("LP liq resycle")
lpResycle.addStream(lpLiqmixer.getOutStream())
lpResycle.setOutletStream(oilThirdStageToSep)

exportCompressor1 = compressor(dewPointScrubber.getGasOutStream())
exportCompressor1.setName("export 1st stage")
exportCompressor1.setOutletPressure(140.0)
exportCompressor1.setIsentropicEfficiency(0.75)

exportInterstageCooler = cooler(exportCompressor1.getOutStream())
exportInterstageCooler.setName("interstage stage cooler")
exportInterstageCooler.setOutTemperature(inputdata['firstStageExportCoolerTemperature'],'C')

exportCompressor2= compressor(exportInterstageCooler.getOutStream())
exportCompressor2.setName("export 2nd stage")
exportCompressor2.setOutletPressure(200.0)
exportCompressor2.setIsentropicEfficiency(0.75)

exportCooler = cooler(exportCompressor1.getOutStream())
exportCooler.setName("export cooler")
exportCooler.setOutTemperature(inputdata['secondStageExportCoolerTemperature'],'C')

exportGas = stream(exportCooler.getOutStream())
exportGas.setName("export gas")

oilprocess = getProcess()


# Start simulation

In [4]:
#thread = oilprocess.runAsThread()
oilprocess.run()

# Check if simulation has converged/stopped

True if still running.
False if finished.

In [5]:
#thread.isAlive() 

# Read results from simulation

In [6]:
TVP = stableOil.TVP(20.0, 'C')
cricondenbar = exportGas.CCB('bara')
powerComp1 = oilprocess.getUnit("1st stage compressor").getPower()/1.0e3
powerComp2 = oilprocess.getUnit("2nd stage compressor").getPower()/1.0e3
powerExpComp1 = oilprocess.getUnit("export 1st stage").getPower()/1.0e3
powerExpComp2 = oilprocess.getUnit("export 2nd stage").getPower()/1.0e3

print('TVP [bara] ', TVP)
print('cricondenbar [bara] ', cricondenbar)
print('recomp power 1 [kW] ', powerComp1)
print('recomp power 2 [kW] ', powerComp2)
print('exp power 1 [kW] ', powerExpComp1)
print('exp power 2 [kW] ', powerExpComp2)


TVP [bara]  1.0172395402888048
cricondenbar [bara]  86.7341553332676
recomp power 1 [kW]  63.25679123907593
recomp power 2 [kW]  71.84636683376594
exp power 1 [kW]  8677.903943983109
exp power 2 [kW]  4475.98774288613


In [7]:
from neqsim.thermo import fluid, printFrame
printFrame(waterSaturatedFeedGas.getFluid())

                          total         gas          oil                     
           nitrogen  7.94066E-4  8.11334E-4   2.12556E-4      [mole fraction]
                CO2  3.53359E-2  3.56464E-2   2.48808E-2      [mole fraction]
            methane   8.6712E-1  8.80525E-1   4.15706E-1      [mole fraction]
             ethane  3.99018E-2  3.99399E-2   3.86186E-2      [mole fraction]
            propane  1.52858E-2  1.50306E-2     2.388E-2      [mole fraction]
           i-butane  1.98516E-3  1.91844E-3   4.23211E-3      [mole fraction]
           n-butane  4.16884E-3  3.98566E-3   1.03376E-2      [mole fraction]
          i-pentane  1.48887E-3  1.38289E-3   5.05786E-3      [mole fraction]
          n-pentane  1.98516E-3   1.8232E-3   7.43943E-3      [mole fraction]
              C6_PC   2.3822E-3  2.05829E-3   1.32898E-2      [mole fraction]
              C7_PC  3.37478E-3  2.74265E-3   2.46619E-2      [mole fraction]
              C8_PC  3.27552E-3   2.4597E-3   3.07486E-2      [m

# Save process to file

In [8]:
oilprocess.save('oilpro.neqsim')

# Calculation of heat exchanger area


In [9]:
import math
U_Wm2K = 100.0

water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('1st stage cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('1st stage cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('1st stage cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))

Q = -oilprocess.getUnit('1st stage cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD ", round(LMTD,3), " degC")
print("heat exchanger duty ", round(oilprocess.getUnit('1st stage cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")

LMTD  10.149  degC
heat exchanger duty  -0.048  MW
heat exchanger area  47.485  m^2
water circulation rate  1.149  m3/hr
